In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,                # Enable LaTeX text rendering
    "font.family": "serif",             # Use a serif font
    "font.serif": ["Computer Modern"],  # Set font to Computer Modern (LaTeX default)
})
sys.path.append('../src/')
from Biologging_Toolkit.applications.Wind import Wind

#### Enter deployment ID and path

Finalized dataset will be saved or loaded from path

In [ ]:
depids = ['ml17_280a','ml18_296a','ml18_294b','ml19_292a','ml19_292b','ml19_293a','ml19_294a','ml20_293a','ml20_296b','ml20_313a','ml21_295a','ml21_305b']
path = 'D:/individus_brut/individus'

In [ ]:
freqs = [50, 100, 200, 300, 500, 1000, 2500, 5000, 8000, 10000, 12000, 15000, 16000]

### PLOT DISTRIBUTIONS

In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
fig, ax = plt.subplots(4, 3, sharex=True, sharey=True, figsize=(6, 6))
ax = ax.flatten()
for i, depid in enumerate(depids):
    df = pd.read_csv(f'D:/individus_brut/individus/{depid}/{depid}_dive.csv')
    _df = df[['upwards_mean_5000', 'downwards_mean_5000', 'wind_speed']]
    _df = _df.melt(id_vars='wind_speed').dropna()
    _df.columns = ['Wind speed (m/s)', 'variable', r'PSD at 5kHz (dB re 1 $\mu$Pa)']
    sns.kdeplot(
        data=_df,
        x='Wind speed (m/s)',
        y=r'PSD at 5kHz (dB re 1 $\mu$Pa)',
        hue="variable",
        ax=ax[i],
        legend = False)
    ax[i].set_xlabel('')
    ax[i].set_ylabel('')
fig.text(0.56, 0.04, 'Wind speed (m/s)', ha='center', va='center', fontsize=14)
fig.text(0.04, 0.5, r'PSD at 5kHz (dB re 1 $\mu$Pa)', ha='center', va='center', rotation='vertical', fontsize=14)
fig.tight_layout(rect=[0.05, 0.05, 1, 1])

In [ ]:
fig.savefig('C:/Users/grosm/Pictures/thèse/SPL_wind_distribution.pdf')

#### Get estimations from different models

In [ ]:
data = np.load('data_frequency.npy')

In [ ]:
data = np.full((12,13,2), np.nan)
for j, freq in enumerate(freqs):
    for i, depid in enumerate(depids) :
        inst = Wind(depids, path = [os.path.join(path, depid) for depid in depids], data = f'downwards_mean_{int(freq)}', test_depid = depid)
        inst.depid_fit()
        data[i,j,0] = inst.wind_model_stats['depid_mae']
        inst = Wind(depids, path = [os.path.join(path, depid) for depid in depids], data = f'downwards_mean_{int(freq)}', test_depid = depid, method = 'Hildebrand')
        inst.depid_fit()
        data[i,j,1] = inst.wind_model_stats['depid_mae']

In [ ]:
fig, ax = plt.subplots(1,2, sharex = True, sharey = True, figsize = (6,5))
colors = ['#1f77b4', '#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2',  '#7f7f7f','#bcbd22','#17becf','#ffbb78','#98df8a']
for i in range(12):
    ax[0].plot(freqs, data[i,:,0], '--o', color = colors[i], alpha = 0.4)
    ax[1].plot(freqs, data[i,:,1], '--o', color = colors[i], label = depids[i], alpha = 0.4)
ax[0].plot(freqs, np.mean(data[:,:,0], axis = 0), linewidth = 3, color = 'crimson', label = 'average mae')
ax[1].plot(freqs, np.mean(data[:,:,1], axis = 0), linewidth = 3, color = 'crimson')
ax[0].legend()
ax[0].grid()
ax[1].grid()
ax[1].set_xlabel('Frequency (Hz)')
ax[0].set_xlabel('Frequency (Hz)')
ax[0].set_ylabel('Mean Absolute Error (m/s)')
fig.tight_layout()

In [ ]:
fig.savefig('C:/Users/grosm/Pictures/thèse/frequency_analysis.pdf')

### Get model performance

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
freq = 5000
for depid in depids:
        inst = Wind(depids, path = [os.path.join(path, depid) for depid in depids], 
                    data = f'dive_mean_{int(freq)}', 
                    test_depid = depid, method = 'Hildebrand')
        inst.depid_fit()
        df[depid] = inst.wind_model_stats
df.to_csv(f'C:/Users/grosm/Desktop/thèse/Results/logarithmic_dive_{freq}.csv', index = None)

In [ ]:
inst.wind_model_stats

In [ ]:
log_up = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/logarithmic_upwards_5000.csv')
log_down = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/logarithmic_downwards_5000.csv')
quad_up = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/quadratic_upwards_5000.csv')
quad_down = pd.read_csv('C:/Users/grosm/Desktop/thèse/Results/quadratic_downwards_5000.csv')

In [ ]:
metric = pd.DataFrame({'up quadratic' : quad_up.mean(axis = 1), 
                       'down quadratic' : quad_down.mean(axis = 1),
                       'up logarithmic' : log_up.mean(axis = 1), 
                       'down logarithmic' : log_down.mean(axis = 1)})

In [ ]:
metric = metric.T
metric.columns = ['mae', 'rmse', 'r1', 'var', 'std']
metric = metric.round(2)
metric = metric.map(lambda x: f"{x:.2f}")
metric.to_latex('C:/Users/grosm/Desktop/thèse/Results/empirical_metric.tex', columns = ['mae', 'rmse', 'std'])

In [ ]:
metric